In [1]:
from hapiclient import hapi, hapitime2datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import calendar
from geopack import geopack
import datetime as datetime

Load IGRF coefficients ...


In [2]:
def fetch_omni_data(params,data, start, stop):
    server      = 'https://cdaweb.gsfc.nasa.gov/hapi'
    dataset     = data
    parameters  = params
    data, meta  = hapi(server, dataset, parameters, start, stop)

    return data

In [3]:
def extract_data(data, params):

    header = params.split(',')
    header.insert(0, 'Timestamp')

    #extract the data from the list
    extracted_data = [[x[i] for x in data] for i in range(len(header))]

    #create a dictionary and convert to dataframe using param names as headers
    df_dict = {header: values for header, values in zip(header, extracted_data)}
    df = pd.DataFrame(df_dict).set_index('Timestamp')

    #convert to datetime from '\b' time
    df.index = hapitime2datetime(df.index.values.astype(str))
    df.index = df.index.tz_convert("UTC").tz_convert(None)


    return df

In [4]:
def extract_omni_1min(data, params):

    df = extract_data(data, params)
    df['Timestamp_1min_omni'] = df.index
    df['Timestamp_1hr_omni'] = pd.to_datetime(df['Timestamp_1min_omni'], errors='coerce').dt.floor('1h')
    df = df.reset_index(drop=True)
    
    return df


In [5]:
def extract_omni_1hr(params, data):

    df = extract_data(data, params)

    df['Timestamp_1hr_omni'] = df.index
    df['Timestamp_1hr_omni'] = pd.to_datetime(df['Timestamp_1hr_omni'], errors='coerce').dt.floor('1h')

    df = df.reset_index(drop=True)
    
    return df

In [7]:
omni_1min = "OMNI_HRO2_1MIN"
#omni_1min_params = 'percent_interp,BY_GSE,BZ_GSE,flow_speed,proton_density,T,Pressure,E,Mach_num,BSN_x,BSN_y,BSN_z,SYM_H'
omni_1min_params = 'BY_GSE,BZ_GSE,flow_speed,proton_density'


omni_1hr = "OMNI2_H0_MRG1HR"
omni_1hr_params = 'F10_INDEX1800,KP1800'

#Pressure = flow pressure in nPa

#yr = 2021
months = []
years = [2013]
for year in years:
    for month in range(5,6):

        print('Processing: ',year, month)

        _, num_days = calendar.monthrange(year, month)
        #num_days = 2

        start_time = f'{year}-{month:02d}-{1:02d}T00:00:000Z'
        end_time = f'{year}-{month:02d}-{num_days:02d}T23:59:59Z'

        #download the data then clean it
        omni_1min_data = fetch_omni_data(omni_1min_params, omni_1min, start_time, end_time)
        omni_1min_df = extract_omni_1min(omni_1min_data, omni_1min_params)

        omni_1hr_data = fetch_omni_data(omni_1hr_params,omni_1hr, start_time, end_time)
        omni_1hr_df = extract_omni_1hr(omni_1hr_params, omni_1hr_data)

        df = pd.merge(omni_1min_df, omni_1hr_df, on='Timestamp_1hr_omni', how='left')
        df = df.rename(columns={'F10_INDEX1800':'F10.7', 'KP1800':'Kp'})
        months.append(df)

months_df = pd.concat(months)

def calculate_tilt_angle(row):
    t0 = datetime.datetime(1970, 1, 1)
    t1 = row['Timestamp_1min_omni']
    tdiff = (t1 - t0).total_seconds()
    tilt_angle = geopack.recalc(tdiff) # Calculate dipole tilt angle
    
    return tilt_angle

#months_df['tilt_angle'] = months_df.apply(calculate_tilt_angle, axis=1)

months_df = months_df.reset_index(drop=True)

#export_path = f'/Users/sr2/My Drive/Career/Employment/Current/JSPS/Research/Analysis/Apr-24/data/omni/' #macbook
#export_path = f'/home/ryuho/Documents/reddy/research/SMRAI/Data/OMNI/' #linux
export_path = f'/home/sachin/Documents/NIPR/Research/Data/OMNI/' #server

df_name = f'omni_hro_1min_2013-05'
export_filename = export_path + df_name +'.csv'
months_df.to_csv(export_filename, index=False, header=True)
months_df
        

Processing:  2013 5


,BY_GSE,BZ_GSE,flow_speed,proton_density,Timestamp_1min_omni,Timestamp_1hr_omni,F10.7,Kp
0,-6.47,-0.88,99999.900000,999.990000,2013-05-01 00:00:00,2013-05-01 00:00:00,161.699997,27
1,-4.74,-1.56,99999.900000,999.990000,2013-05-01 00:01:00,2013-05-01 00:00:00,161.699997,27
2,-7.73,0.18,99999.900000,999.990000,2013-05-01 00:02:00,2013-05-01 00:00:00,161.699997,27
3,-4.49,-1.52,99999.900000,999.990000,2013-05-01 00:03:00,2013-05-01 00:00:00,161.699997,27
4,-6.10,-0.34,380.299988,3.460000,2013-05-01 00:04:00,2013-05-01 00:00:00,161.699997,27
...,...,...,...,...,...,...,...,...
44635,-7.16,6.36,400.200012,29.990000,2013-05-31 23:55:00,2013-05-31 23:00:00,104.699997,37
44636,-6.84,6.68,99999.900000,999.990000,2013-05-31 23:56:00,2013-05-31 23:00:00,104.699997,37
44637,9999.99,9999.99,400.600006,28.120001,2013-05-31 23:57:00,2013-05-31 23:00:00,104.699997,37
44638,-7.19,6.42,400.600006,28.120001,2013-05-31 23:58:00,2013-05-31 23:00:00,104.699997,37


In [9]:
#import_path=r'/Users/sr2/My Drive/Career/Employment/Current/JSPS/Research/Analysis/Apr-24/data/omni/omni_hro_5min_2021-2022.csv'
import_path = r'omni_hro_1min_2013-05.csv'
omni_df = pd.read_csv(import_path)
omni_df

FileNotFoundError: [Errno 2] No such file or directory: 'omni_hro_1min_2013-05.csv'

In [14]:
def clean_omni(df):
    df = df.copy()
    df = df.dropna()

    #filters
    df = df[df['BY_GSE'] < 100]
    df = df[df['BZ_GSE'] < 100]
    df = df[df['flow_speed'] < 1400]
    df = df[df['proton_density'] < 150]
    #f = df[df['T'] < 1e6]
    #df = df[df['Pressure'] < 100]
    #f = df[df['E'] < 100]
    #df = df[df['Mach_num'] < 100]
    #df = df[df['SYM_H'].between(-1000,100)]
    #df = df[df['F10.7'] < 400]
    #df = df[df['Kp'] < 100]
    #df['Kp'] = df['Kp'] / 10

    #change data types
    df[['F10.7','flow_speed']] = df[['F10.7','flow_speed']].astype(int)

    #new features
    df['Timestamp_1min_omni'] = pd.to_datetime(df['Timestamp_1min_omni'])
    df['doy'] = df['Timestamp_1min_omni'].dt.dayofyear
    
    def calculate_tilt_angle(row):
        t0 = datetime.datetime(1970, 1, 1)
        t1 = row['Timestamp_1min_omni']
        tdiff = (t1 - t0).total_seconds()
        tilt_angle = geopack.recalc(tdiff) # Calculate dipole tilt angle
        
        return tilt_angle
    
    df['tilt_angle'] = df.apply(calculate_tilt_angle, axis=1)

    #interpolate missing values
    #df = df.interpolate()
    #df = df.drop(columns=['percent_interp','Timestamp_1hr_omni','BSN_y','BSN_z'])

    #match SMRAI2 inputs
    #df = df.drop(columns=['BSN_x','SYM_H','doy','T','Pressure','E','Mach_num','SYM_H','F10.7','Kp']).
    df['dt'] = df['Timestamp_1min_omni']
    df['dt'] = pd.to_datetime(df['dt'])
    df = df[['dt','BY_GSE','BZ_GSE','flow_speed','proton_density','tilt_angle']]
    return df

omni_df = clean_omni(months_df)
omni_df.to_csv(export_filename, index=False, header=True)

In [ ]:
#show nan values


In [ ]:
#Read MHD dates and expand-out the date ranges
mhd_data = pd.read_csv('mhd_dates.csv')
expanded_dt = pd.concat([pd.Series(pd.date_range(start, end)) 
        for start, end in zip(mhd_data['start'], mhd_data['end'])])

time = np.arange(288) #24hrs in 5min intervals
dt = [] 
for day in expanded_dt:
    for t in time:
        dt.append(day + pd.Timedelta(minutes=t*5)) #add 5min intervals to each day
dt = np.array(dt) #convert from list to numpy array
mhd_dates = pd.DataFrame({'Timestamp_1min_omni': dt}) 
mhd_dates

In [ ]:
omni_mhd_dt_merged = pd.merge(mhd_dates, omni_df, on='Timestamp_1min_omni', how='left')
omni_mhd_dt_merged.rename(columns={'Timestamp_1min_omni':'dt'}, inplace=True)
omni_mhd_dt_merged.sort_values(by='dt', inplace=True)
#omni_mhd_dt_merged =omni_mhd_dt_merged.interpolate()
#check for nan values
omni_mhd_dt_merged.isnull().sum()

In [ ]:
# Merge the dataframes
omni_mhd_dt_merged = pd.merge(mhd_dates, omni_df, on='Timestamp_1min_omni', how='left')
omni_mhd_dt_merged.rename(columns={'Timestamp_1min_omni': 'dt'}, inplace=True)
omni_mhd_dt_merged.sort_values(by='dtinterpolated
', inplace=True)

# Interpolate missing values
omni_mhd_dt_merged['interpolated'] = omni_mhd_dt_merged.isnull().any(axis=1).astype(int)
omni_mhd_dt_merged = omni_mhd_dt_merged.interpolate()

# Plotting
fig, axes = plt.subplots(nrows=len(omni_mhd_dt_merged.columns[1:-1]), ncols=1, figsize=(10, 2 * len(omni_mhd_dt_merged.columns[1:-1])), sharex=True)
for i, column in enumerate(omni_mhd_dt_merged.columns[1:-1]):
    axes[i].plot(omni_mhd_dt_merged['dt'], omni_mhd_dt_merged[column], label=column)
    axes[i].scatter(omni_mhd_dt_merged['dt'][omni_mhd_dt_merged['interpolated'] == 1], omni_mhd_dt_merged[column][omni_mhd_dt_merged['interpolated'] == 1], color='red', label='Interpolated')
    axes[i].set_ylabel(column)
    axes[i].legend()
    axes[i].grid(True)
plt.xlabel('Date')
plt.tight_layout()
plt.show()


In [ ]:
#export omni_mhd_dt_merged
#export_path = f'/Users/sr2/My Drive/Career/Employment/Current/JSPS/Research/Analysis/Apr-24/data/omni/'
#export_path = f'/home/sachin/Documents/NIPR/Research/VSCode/REPPU-ESN2/SR_ML/'
df_name = f'omni_add-feats_mhd_5min'
#export_filename = export_path + df_name +'.csv'
export_filename = df_name +'.csv'
omni_mhd_dt_merged.to_csv(export_filename, index=False, header=True)


In [ ]:
#show nan values
nan_values = omni_mhd_dt_merged[omni_mhd_dt_merged.isna().any(axis=1)]
nan_values.sort_values(by='dt')

Plots are below. Not part of main code

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
omni_df.hist(ax=ax)
plt.tight_layout()

In [ ]:
y = omni_df['doy']
x = omni_df['tilt_angle']
r2 = x.corr(y)
sns.jointplot(x=x, y=y, kind='scatter', s=1, height=4.5)
RE = r'R$_E$'
pcc = r'cm$^{-3}$'
#plt.xlabel(f'Proton Density [{pcc}]')
#plt.ylabel(f'Bow Shock Nose Location (GSE-X) [{RE}]')
#plt.xlabel('Pressure [nPa]')

#plt.ylabel('Electric Field [mV/m]')
#plt.xlabel('Bz [nT]')

r2_lab = r'R$^2$'    
annotation = f'{r2_lab} = {r2:.2f}'
plt.annotate(annotation, xy=(0.75, 0.7), xycoords='axes fraction', fontsize=11)
plt.tight_layout()
plt.savefig('/Users/sr2/OneDrive - University College London/PhD/Experiences/Postdocs/JSPS/Research/Analysis/Apr-24/plots/omni/R2_tilt-angle_doy.png', dpi=300)

In [ ]:
corr = omni_df.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(7.5, 6.5))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap='coolwarm', vmin=-1, vmax=1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .7}, annot=True,
             fmt=".2f")

plt.title('OMNI 5min Data Correlation Matrix \n January 2021 - December 2022',pad=-40)
plt.tight_layout()
#plt.savefig('/Users/sr2/OneDrive - University College London/PhD/Experiences/Postdocs/JSPS/Research/Analysis/Apr-24/plots/omni/omni_heatmap.png', dpi=400)

In [ ]:
sns.histplot(omni_mhd_dt_merged['flow_speed'], bins=50)
omni_mhd_dt_merged['flow_speed'].describe()

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(omni_df.corr(), annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0, fmt=".2f")